In [14]:
from tqdm import tqdm
import pandas as pd
import random


pd.set_option("display.precision",16 )
train_path='../data/underexpose_train/'
test_path='../data/underexpose_test/'
now_p=6

col_name=['item_id']
for i in range(0,128):
    col_name.append('tv'+str(i))
for i in range(0,128):
    col_name.append('iv'+str(i))



itemft=pd.read_csv(train_path+'underexpose_item_feat.csv',low_memory=False,names=col_name)
itemft=itemft.replace('[\[\]]','',regex=True)#regex=True is the key
#underexpose_item_feat.apply(lambda x: x.str.replace('[\[\]]',''),axis=1)
userft=pd.read_csv(train_path+'underexpose_user_feat.csv',low_memory=False,names=['user_id','age','gender','city'],dtype=\
                  {'user_id': np.int32,'age':'category','gender':'category','city':'category'})


In [81]:
for p in tqdm(range(0,now_p+1)):


    tnc=pd.read_csv(train_path+r'underexpose_train_click-{}.csv'.format(p),low_memory=False,names=['user_id','item_id','time'])
    ttc=pd.read_csv(test_path+r'underexpose_test_click-{}/underexpose_test_click-{}.csv'.format(p,p),low_memory=False\
                   ,names=['user_id','item_id','time'])

    # ttt=pd.read_csv(test_path+'utc_click-{}.csv'.format(p),low_memory=False)
    # ttt.drop(ttt.columns[0],axis=1,inplace=True) # !!
    a=userft.user_id.value_counts()
    dup_user_id=a[a>1].index.to_list()
    dup_index=userft.loc[userft.user_id.isin(dup_user_id),:].index.to_list()
    dup_index=[i for i in dup_index if i%2 == 0]
    userft.drop(dup_index,inplace=True)
    temp=tnc.merge(itemft,how='left')
    assert tnc.shape[0]==temp.shape[0]

    all_tn=temp.merge(userft,left_on=['user_id'],right_on=['user_id'],how='left')
    assert temp.shape[0] == all_tn.shape[0]
    all_tn.to_csv(train_path+'train_{}.csv'.format(p),index=False)

100%|██████████| 7/7 [24:43<00:00, 211.94s/it]


## get matrix

In [4]:
from tqdm import tqdm
import pandas as pd
train_path='../data/underexpose_train/'
test_path='../data/underexpose_test/'
now_p=6
df=pd.read_csv('../user_data/'+'train_{}.csv'.format(now_p),low_memory=False)

In [4]:
item_user_matrix=df.pivot_table(index='item_id',columns='user_id',aggfunc='size',fill_value=0)

In [5]:
item_user_matrix.shape

(48662, 18575)

In [11]:
item_user_matrix_min=item_user_matrix.dropna()

In [9]:
item_user_matrix.to_hdf(train_path+'item_user_matrix.h5',key='result',index=False)

HDF5ExtError: Problems creating the Array.

In [14]:
df.shape

(310372, 262)

In [15]:
df.dropna().shape

(71688, 262)

## get non-null database

In [16]:
df_min=df.dropna()

In [17]:
df_min.to_csv(train_path+'train_min_{}'.format(now_p),index=False)

In [5]:
df_min=pd.read_csv('../user_data/'+'train_min_{}'.format(now_p))

In [6]:
database=df_min.copy().drop(['user_id','item_id'],axis=1)
database=database.replace({'M':1,'F':0})
database=database.astype({'age': 'int','gender':'int','city':'int'})
database.head(3)

,time,tv0,tv1,tv2,tv3,tv4,tv5,tv6,tv7,tv8,...,iv121,iv122,iv123,iv124,iv125,iv126,iv127,age,gender,city
0,0.984111,4.148138,-0.213997,-0.274804,0.394361,-2.207227,-1.437607,2.098062,-0.612489,-4.826493,...,-2.937082,-0.981116,0.425762,1.491055,3.889630,1.439132,-0.080906,4,0,4
1,0.984263,4.148138,-0.213997,-0.274804,0.394361,-2.207227,-1.437607,2.098062,-0.612489,-4.826493,...,-2.937082,-0.981116,0.425762,1.491055,3.889630,1.439132,-0.080906,4,0,2
2,0.984069,5.053555,1.216570,0.348558,0.380879,5.722718,-0.903466,-2.930219,-2.049398,-7.384793,...,-1.356675,-1.060216,0.303131,1.582733,1.423042,2.876045,-0.571884,3,0,5


## true value

In [9]:
c=6
temp=pd.read_csv(test_path + '/underexpose_test_click-{}/underexpose_test_click-{}.csv'.format(c,c), header=None,  names=['user_id', 'item_id', 'time'])
temp=temp.sort_values(['user_id','time'],ascending=[True,False]).groupby(['user_id']).agg('max').reset_index()
query=temp.merge(itemft,how='left').merge(userft,how='left').drop(['user_id','item_id'],axis=1)
query=query.replace({'M':1,'F':0})
query=query.astype({'age': 'int','gender':'int','city':'int'},errors='ignore')
query.head(3)

query.info()
query.shape
query.select_dtypes(['category'])
query['age'].dtype =='category'

database['age'].median()

summary=database.mean()
for col in query.isna().any().loc[query.isna().any() == True].index:
    if col in ['age','gender','city']:
        a=summary[col]
    else:
        a=database[col].median()
    query[col]=query[col].replace(np.nan,a)
query.shape == query.dropna().shape

query.head()
database.head()

from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
query1=ss.fit_transform(query)
database1=ss.fit_transform(database)

,time,tv0,tv1,tv2,tv3,tv4,tv5,tv6,tv7,tv8,...,iv121,iv122,iv123,iv124,iv125,iv126,iv127,age,gender,city
0,0.9842771241305298,2.101147174835205,-0.8465766906738281,0.0573386922478676,-3.1112377643585205,-0.8484025001525879,2.9340057373046875,1.1302022933959959,-0.5584614276885986,-2.9435551166534424,...,1.5269055366516111,-2.7677159309387207,-0.5956848263740540,0.1572125703096390,1.0126974582672121,2.7917819023132324,0.3088803291320801,8,1.0,4
1,0.9842517769474944,1.900734305381775,2.1579475402832031,3.0377004146575928,-3.0554065704345703,1.2677739858627319,-1.0928924083709717,-3.7414407730102539,-1.1887992620468140,-3.2973101139068604,...,1.0075087547302246,-1.0377058982849119,0.3403505682945251,2.2763986587524414,2.3016302585601807,1.0346159934997561,-0.8432886004447937,NaN,NaN,NaN
2,0.9842692676835492,1.830065131187439,1.8811055421829224,2.6448986530303955,-2.4803488254547119,-1.0310925245285034,1.9788519144058228,-1.2319024801254272,-1.8476966619491575,-3.5557911396026611,...,-0.2710131108760834,0.2487333118915558,2.4548890590667725,-0.1799300462007523,2.5021350383758545,0.2804744839668274,0.44003647565841675,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1821 entries, 0 to 1820
Columns: 260 entries, time to city
dtypes: category(2), float64(254), object(4)
memory usage: 3.6+ MB


(1821, 260)

,age,city
0,8,4
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,6,1
...,...,...
1816,NaN,NaN
1817,NaN,NaN
1818,NaN,NaN
1819,NaN,NaN


True

4.0

True

,time,tv0,tv1,tv2,tv3,tv4,tv5,tv6,tv7,tv8,...,iv121,iv122,iv123,iv124,iv125,iv126,iv127,age,gender,city
0,0.9842771241305298,2.101147174835205,-0.8465766906738281,0.0573386922478676,-3.1112377643585205,-0.8484025001525879,2.9340057373046875,1.1302022933959959,-0.5584614276885986,-2.9435551166534424,...,1.5269055366516111,-2.7677159309387207,-0.5956848263740540,0.1572125703096390,1.0126974582672121,2.7917819023132324,0.3088803291320801,8,1.0000000000000000,4
1,0.9842517769474944,1.900734305381775,2.1579475402832031,3.0377004146575928,-3.0554065704345703,1.2677739858627319,-1.0928924083709717,-3.7414407730102539,-1.1887992620468140,-3.2973101139068604,...,1.0075087547302246,-1.0377058982849119,0.3403505682945251,2.2763986587524414,2.3016302585601807,1.0346159934997561,-0.8432886004447937,4.248995647807164,0.1149425287356322,3.674087713424841
2,0.9842692676835492,1.830065131187439,1.8811055421829224,2.6448986530303955,-2.4803488254547119,-1.0310925245285034,1.9788519144058228,-1.2319024801254272,-1.8476966619491575,-3.5557911396026611,...,-0.2710131108760834,0.2487333118915558,2.4548890590667725,-0.1799300462007523,2.5021350383758545,0.2804744839668274,0.44003647565841675,4.248995647807164,0.1149425287356322,3.674087713424841
3,0.9842643606374868,1.8137321472167969,1.3914031982421875,2.1369509696960449,-4.1689491271972665,-2.8606204986572266,1.9786089658737185,-0.2026271671056747,-1.7621158361434937,-3.4887967109680176,...,-1.4988899230957031,1.6837480068206787,1.1026927232742310,-0.2537772655487061,2.9432177543640137,1.3797429800033569,-0.885633647441864,4.248995647807164,0.1149425287356322,3.674087713424841
4,0.9842105925855824,0.8316391706466675,1.8352080583572388,1.0361447334289549,-3.9268870353698730,-0.4585402011871338,0.8195793628692627,3.3453412055969238,-4.2287759780883789,-3.3158850669860840,...,1.4824553728103638,4.6624727249145508,0.7338652014732361,1.7606782913208008,1.0468974113464355,0.5324596166610718,-0.70954829454422,6,1.0000000000000000,1


,time,tv0,tv1,tv2,tv3,tv4,tv5,tv6,tv7,tv8,...,iv121,iv122,iv123,iv124,iv125,iv126,iv127,age,gender,city
0,0.9841110719125040,4.1481380462646484,-0.2139974832534789,-0.2748044133186340,0.3943611383438110,-2.2072267532348642,-1.4376068115234375,2.0980615615844727,-0.6124892234802246,-4.8264927864074716,...,-2.9370822906494136,-0.9811164140701294,0.4257615506649017,1.4910547733306885,3.8896303176879878,1.4391320943832395,-0.0809062123298645,4,0,4
1,0.9842627512424618,4.1481380462646484,-0.2139974832534789,-0.2748044133186340,0.3943611383438110,-2.2072267532348642,-1.4376068115234375,2.0980615615844727,-0.6124892234802246,-4.8264927864074716,...,-2.9370822906494136,-0.9811164140701294,0.4257615506649017,1.4910547733306885,3.8896303176879878,1.4391320943832395,-0.0809062123298645,4,0,2
2,0.9840694762116832,5.0535550117492685,1.2165696620941162,0.3485578894615173,0.3808794021606445,5.7227177619934073,-0.9034663438796996,-2.9302191734313965,-2.0493977069854736,-7.3847928047180176,...,-1.3566752672195437,-1.0602157115936279,0.3031310141086578,1.5827329158782959,1.4230424165725708,2.8760449886322017,-0.5718842148780823,3,0,5
3,0.9842570189950302,2.9716651439666748,-0.5788695812225342,2.2469174861907959,-1.7302743196487431,-2.3264360427856445,-0.5912094712257385,-2.3438596725463867,0.4770196080207825,-3.7080161571502690,...,0.9559630155563354,-0.0576305389404297,-2.8185224533081050,-3.0371842384338379,0.8351709246635437,1.1768972873687744,0.4194931983947754,2,0,5
4,0.9840924399002436,0.4042505621910095,-1.0307354927062988,-0.3946554064750672,-1.1291629076004028,1.5390796661376951,0.4129871129989624,-0.1573128253221512,0.0748407095670700,-3.0261974334716801,...,-1.2563368082046509,-1.1090230941772459,1.2311370372772217,1.1668357849121094,3.2568092346191406,2.9602167606353764,-1.0459980964660645,4,0,6


## query records

In [183]:
temp=temp.sort_values(by=['user_id','time'],ascending=[True,False]).groupby(['user_id']).apply(\
                                                        lambda x : x.nlargest(2,'time').min()).drop(['user_id'],axis=1).reset_index()
temp.head(3) # GET the second last click

query=temp.merge(itemft,how='left').merge(userft,how='left').drop(['user_id','item_id'],axis=1)
query=query.replace({'M':1,'F':0})
query=query.astype({'age': 'int','gender':'int','city':'int'},errors='ignore')
query.head(3)

query.info()
query.shape
query.select_dtypes(['category'])
query['age'].dtype =='category'

database['age'].median()

summary=database.mean()
for col in query.isna().any().loc[query.isna().any() == True].index:
    if col in ['age','gender','city']:
        a=summary[col]
    else:
        a=database[col].median()
    query[col]=query[col].replace(np.nan,a)
query.shape == query.dropna().shape

query.head()
database.head()

from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
query1=ss.fit_transform(query)
database1=ss.fit_transform(database)

## find similarity

In [16]:
user_path='../user_data/'

from numpy import genfromtxt
database1 = genfromtxt(user_path+'database1.csv', delimiter=',')
query1 = genfromtxt(user_path+'query1.csv', delimiter=',')
query1


array([[-1.65853636,  0.42432929, -1.06798817, ..., -0.13081972,
        -0.20516912, -0.15768494],
       [-1.65843807, -0.46582549, -0.11851271, ..., -0.13081972,
        -0.20516912, -0.15768494],
       [-1.65833978, -0.20378177,  0.36148371, ..., -0.13081972,
        -0.20516912, -0.15768494],
       ...,
       [ 1.82410461,  0.99336521,  0.37027228, ..., -0.13081972,
        -0.20516912, -0.15768494],
       [ 1.82420289,  1.30946385,  1.13300477, ..., -0.13081972,
        -0.20516912, -0.15768494],
       [ 1.82430118,  1.75579629,  1.54891634, ..., -0.13081972,
        -0.20516912, -0.15768494]])

In [20]:
user_id=genfromtxt(user_path+'user_id.csv',delimiter=',').reshape(-1,1)

In [21]:
database1.shape
query1.shape
user_id.shape

(5984, 260)

(12081, 262)

(12081, 1)

In [22]:
query1=np.delete(query1,[0,1],1)

In [23]:
import faiss

database1.dtype
database1=np.ascontiguousarray(database1.astype('float32'))
query1=np.ascontiguousarray(query1.astype('float32'))



dim=260
k=50 # recall 10
nlist=100
quantizer = faiss.IndexFlatL2(dim)
idx=faiss.IndexIVFPQ(quantizer,dim,nlist,10,8)
idx.nprobe =10

# index_sim.add(np.ascontiguousarray(tr_features_np))


idx.train(np.ascontiguousarray(database1))
idx.add(np.ascontiguousarray(database1))
d,i=idx.search(np.ascontiguousarray(query1),100)

dtype('float64')

In [24]:
result=np.ndarray(shape=d.shape)

In [35]:
i.shape

(12081, 100)

In [27]:
df_min=pd.read_csv(user_path+'df_min.csv')
df_min.head(3)

,user_id,item_id,time,tv0,tv1,tv2,tv3,tv4,tv5,tv6,...,iv121,iv122,iv123,iv124,iv125,iv126,iv127,age,gender,city
0,10,88804,0.9842770799683772,6.0212249755859375,-4.5798912048339844,3.2130193710327148,-2.1399679183959961,-0.9064429402351379,2.4552919864654541,0.9194136857986450,...,-1.8558883666992188,1.1678773164749146,1.3769862651824951,-3.4025032520294189,1.5410439968109131,3.6849513053894039,2.9092373847961426,6,F,1
1,13,42888,0.9842780035882582,-0.6111484169960022,-0.1359649300575257,3.6519391536712646,-1.3654018640518188,0.9543317556381226,-1.1697646379470823,2.4903392791748047,...,-2.1855368614196777,1.3929390907287598,-0.0513036102056503,0.0396473109722138,2.7789900302886963,2.3467061519622803,-0.6091269850730896,7,M,6
2,16,73359,0.9842585274479918,-0.1482433378696441,3.3703970909118652,1.3680076599121094,-1.4550262689590454,2.8250017166137682,-2.0104930400848398,-4.0500035285949716,...,-1.3966966867446899,-0.9993148446083068,0.7488551735877991,1.6849117279052734,3.7666616439819336,2.0998668670654297,-2.2072551250457764,6,F,2


In [28]:
df_min.shape

(5984, 262)

In [29]:
for row_i in tqdm(range(len(d))):
    result[row_i]=df_min.loc[i[row_i],'item_id'].values

100%|██████████| 12081/12081 [00:12<00:00, 961.08it/s] 


In [40]:
result

array([[ 69359.,  50909.,   5099., ...,  10255.,  93590.,  47448.],
       [ 51505.,  92384.,  18071., ..., 101172.,  30208.,  90430.],
       [ 72872., 106214.,  72108., ...,  92873.,  63028.,  25356.],
       ...,
       [ 23012.,  26666., 102702., ...,  23614.,  69029.,   7264.],
       [101786.,  55365., 108477., ..., 107396.,  87163.,  58509.],
       [108744., 111098., 111189., ..., 107032.,  52206., 115323.]])

In [42]:

resultdf=pd.DataFrame(result,columns=['col'+ str(i) for i in range(0,100)])

In [44]:
resultdf.head()

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,...,col90,col91,col92,col93,col94,col95,col96,col97,col98,col99
0,69359.0,50909.0,5099.0,21671.0,4685.0,36760.0,2997.0,46494.0,23409.0,81561.0,...,46335.0,19248.0,27133.0,17019.0,16780.0,48284.0,83784.0,10255.0,93590.0,47448.0
1,51505.0,92384.0,18071.0,67625.0,99693.0,97339.0,81417.0,45621.0,55194.0,52342.0,...,106510.0,40460.0,89179.0,105452.0,49125.0,75373.0,113694.0,101172.0,30208.0,90430.0
2,72872.0,106214.0,72108.0,92002.0,77056.0,8207.0,8207.0,8207.0,26223.0,62422.0,...,78868.0,63028.0,6969.0,76562.0,76562.0,76562.0,92433.0,92873.0,63028.0,25356.0
3,101786.0,52526.0,55365.0,39885.0,96240.0,108477.0,108477.0,75373.0,88653.0,44114.0,...,81217.0,75182.0,57934.0,46368.0,31323.0,30208.0,110812.0,67910.0,45587.0,101295.0
4,43555.0,53413.0,17460.0,82329.0,113135.0,42193.0,53413.0,113135.0,11933.0,46983.0,...,70204.0,91294.0,105391.0,111639.0,112830.0,28993.0,44775.0,71158.0,86504.0,28571.0


In [45]:
top50_click = df_min['item_id'].value_counts().index[:50].values

top50_click=list(top50_click)

In [46]:
set([2,3,5,7])-set([2,3,6])

{5, 7}

In [48]:
result_filled=np.ndarray(shape=(d.shape[0],50))

sm=0
row_i=0
for item in list(map(set,resultdf.values)):
    diff=50-len(item)
    if diff>=0:
        result_filled[row_i]=list(item)+random.sample(list(set(top50_click)-item),diff)
        if len(list(set(top50_click)-item))<diff:
            print('ooooops')
            random.sample(list(set(top50_click)-item),diff)
        sm+=1
    else:
        result_filled[row_i]=list(item)[:50]
        
    row_i=row_i+1

        
sm
result_filled.shape == d.shape

0

False

In [49]:
result_filled.shape

(12081, 50)

In [51]:
user_id.shape

(12081, 1)

In [50]:
d.shape

(12081, 100)

In [53]:
result_filled=np.concatenate((user_id,result_filled),axis=1)

In [54]:
result_filled.shape

(12081, 51)

In [55]:
np.savetxt('underexpose_submit-6.csv',result_filled,delimiter=',',fmt='%i')